In [5]:
# !pip install matplotlib
#predicting house prices
import pandas as pd
import numpy as np
import matplotlib.pyplot  as plt
from sklearn import linear_model
import math


ModuleNotFoundError: No module named 'pandas'

In [ ]:
hse = pd.read_csv("bengaluru_house_prices.csv")
hse

In [ ]:
print(hse.columns)

In [ ]:
hse.groupby('area_type')['area_type'].agg('count')

In [ ]:
hse2 = hse.drop(['area_type','society','availability','balcony'], axis="columns")
hse2

In [ ]:
hse2.isnull().sum()

In [ ]:
hse3 = hse2.dropna()
hse3

In [ ]:
hse3.isnull().sum()

In [ ]:
hse3.shape

In [ ]:
#This gives us unique values in the column
hse3['size'].unique()

In [ ]:
#create a column containing HBK only
hse3['BHK'] = hse3['size'].apply(lambda x: int(x.split(' ')[0]))

In [ ]:
hse3.head()

In [ ]:
hse3['BHK'].unique()

In [ ]:
hse3[hse3['BHK']>20]

In [ ]:
hse3['total_sqft'].unique()

In [ ]:
def is_float(x):
  try:
    float(x)
  except:
    return False
  return True


In [ ]:
hse3[~hse3['total_sqft'].apply(is_float)] #returns values that are not float

In [ ]:
#We will take the average of the range given to us in the total_sqtf column to get a single value
def convert_sqft_to_num(x):
  tokens = x.split('-')
  if len(tokens) == 2:
    return(float(tokens[0]) + float(tokens[1]))/2
  try:
      return float(x)
  except:
     return None


In [ ]:
hse4 = hse3.copy()
hse4['total_sqft'] = hse4['total_sqft'].apply(convert_sqft_to_num)
hse4.head()

In [ ]:
hse4.loc[122]

In [ ]:
(3067+8156)/2 #average of location 122

In [ ]:
hse5 = hse4.copy()

In [ ]:
#Let's get into the feature engineering by finding the price per square_feet
hse5['price_per_sqft'] = hse5['price'] * 100000/hse5['total_sqft'] #price is in lukh rupee and that's why we multiply by 100000
hse5.head()

In [ ]:
hse5['location'].unique()

In [ ]:
hse5.count()

In [ ]:
len(hse5['location'].unique()) #This are the locations we are dealing with.

In [ ]:
hse5['location'] = hse5['location'].apply(lambda x: x.strip()) # this function strips any extra space in location
# The following variable will give us statistics on the location and sorts the locations by  descending order
location_stats = hse5.groupby('location')['location'].agg('count').sort_values(ascending = False)
location_stats

In [ ]:
len(location_stats[location_stats <=10]) #location with less or  equal than 10 houses

In [6]:
location_stats_less_10 =location_stats[location_stats<=10]
location_stats_less_10

NameError: name 'location_stats' is not defined

In [ ]:
len(hse5['location'] .unique())

In [ ]:
#let's sort out the locations with less than 10 houses
hse5['location'] = hse5['location'].apply(lambda x: 'other' if x in location_stats_less_10 else x)
len(hse5['location'] .unique())

In [ ]:
hse5.head()

In [ ]:
hse5[hse5.total_sqft/hse5.BHK<300].head() #checking  the properties where the threshhold of 300 per unit square is not met

In [ ]:
#Removing outliers(data that falls way outside the typical data)
hse5.shape

In [ ]:
hse6 = hse5[~(hse5.total_sqft/hse5.BHK<300)]
hse6

In [ ]:
hse6.shape

In [ ]:
hse6['price_per_sqft'].describe()

In [ ]:
def remove_outliers(hse):
  hse_out = pd.DataFrame()
  for key,subhse in hse.groupby('location'):
    m = np.mean(subhse.price_per_sqft)
    st = np.std(subhse.price_per_sqft)
    reduced_hse = subhse[(subhse.price_per_sqft> m-st) & (subhse.price_per_sqft<=(m+st))]
    hse_out = pd.concat([hse_out, reduced_hse], ignore_index=True)

  return (hse_out)

hse7 = remove_outliers(hse6)
hse7.shape
#removing outliers(data that fall far out of range)

In [ ]:
hse7.head()

In [ ]:
hse.head()

In [ ]:
#let's create a  function that will help us in visualization
def plot_scatter_chat(hse,location):
  BHK2 = hse[(hse['location']==location) & (hse['BHK'] == 2)]
  BHK3 = hse[(hse['location']==location) & (hse['BHK'] == 3)]
  plt.figure(figsize=(15, 10))
  plt.scatter(BHK2.total_sqft, BHK2.price_per_sqft, color='blue',label='2 BHK', s=50)
  plt.scatter(BHK3.total_sqft, BHK3.price_per_sqft, marker='+', color='green',label='3 BHK', s=50)
  plt.xlabel("Total square feet")
  plt.ylabel("Price per square feet")
  plt.title(location)
  plt.legend()


plot_scatter_chat(hse7,'Assalam ALeikum')


In [ ]:
def remove_outliers(hse):
   exclude_indices = np.array([])
   for location, location_hse in hse.groupby('location'):
       BHK_stats = {}
       for BHK, BHK_hse in location_hse.groupby('BHK'):
        BHK_stats[BHK] = {
            "mean": np.mean(BHK_hse.price_per_sqft),
            "std" : np.std(BHK_hse.price_per_sqft),
            'count': BHK_hse.shape[0],
        }

        for BHK, BHK_hse in location_hse.groupby('BHK'):
            stats = BHK_stats.get(BHK-1)
            if stats and stats['count']>5:
               exclude_indices = np.append(exclude_indices, BHK_hse[BHK_hse.price_per_sqft < (stats['mean'])].index.values)

   return hse.drop(exclude_indices,axis='index')


hse8 = remove_outliers(hse7)
hse8.shape



In [ ]:
plot_scatter_chat(hse8['BHK'],'Assalam ALeikum')

In [ ]:
hse8[hse8.bath>10]

In [ ]:
plt.hist(hse8['bath'], rwidth=0.8)
plt.xlabel("Number of Bathrooms")
plt.ylabel("Count")

In [ ]:
hse8.bath.unique()

In [ ]:
hse8[hse8.bath > hse8.BHK+2]


In [ ]:
hse9 = hse8[hse8.bath < hse8.BHK+2]
hse9.shape
#Our data is now clean!!

In [ ]:
#Let's drop some unnecessarry features for the model building
hse10 = hse9.drop(['price_per_sqft',"size"], axis="columns")
hse10

In [ ]:
#Converts categorical variables (columns containing non-numerical data like text labels) into one-hot encoded dummy variables.
dummies = pd.get_dummies(hse10['location'])
dummies

In [ ]:
#Let's create a new dataframe with dummies inside, We drop one column to avoid hot dummy trap and use 0s in other columns to represent it
hse11 = pd.concat([hse10, dummies.drop('other', axis='columns')] ,axis="columns")
hse11

In [ ]:
hse12 = hse11.drop('location', axis ='columns') #we drop location since it's dummy encoded
hse12

In [ ]:
hse12.shape

In [ ]:
#let's group the independent variables
x = hse12.drop('price', axis='columns')
x

In [ ]:
y = hse12['price']
y

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.2,random_state=10)

In [ ]:
from sklearn.linear_model import LinearRegression
lr_crf = LinearRegression()
lr_crf.fit(x_train, y_train) #trains the model
lr_crf.score(x_test, y_test) #Gives us the performance score of our model


In [ ]:
#we apply k-fold cross-validation(Revisit tutorial)
!pip install scikit-learn

from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_val_score

cv = ShuffleSplit(n_splits=5, test_size = 0.2, random_state=0) #shufflesplit will randomize our samples so as to have equal distributions

cross_val_score(LinearRegression(), x,y, cv=cv)

In [ ]:
#Let's use Grid search cv to find the best possible algorithm to use
from sklearn.model_selection import GridSearchCV

from sklearn.linear_model import Lasso
from sklearn.tree import DecisionTreeRegressor

#function to find best model using gridsearchcv, params represent the parameters
def find_best_model(x,y):
    algos ={
        'linear_regression': {
            'model': LinearRegression(),
            'params': {
              #'normalize': [True,False]
            }
        },
        'lasso': {
            'model': Lasso(),
            'params': {
                'alpha':[1,2],
                'selection':['random', 'cyclic']
            }
        },
        'decision_tree': {
            'model': DecisionTreeRegressor(),
            'params':{
                'criterion': ['mse','friedman_mse'],
                'splitter' : ['best','random']
            }
        }
    }
    scores =[]
    cv = ShuffleSplit(n_splits=5, test_size = 0.2, random_state=0)
    for algo_name ,config in algos.items():
      gs = GridSearchCV(config['model'], config['params'], cv=cv, return_train_score = False)
      gs.fit(x,y)
      scores.append({
          'model': algo_name,
          'best_score':gs.best_score_,
          'best_params': gs.best_params_,
      })

    return pd.DataFrame(scores,columns=['model','best_score','best_params'])

find_best_model(x,y)
#linear_regression is the best with 81% after results

In [ ]:
#Let's make  the predictions for testing
def predict_price(location,sqft,bath,bhk):
    loc_index = np.where(x.columns == location)[0][0]

    xp= np.zeros(len(x.columns))
    xp[0]= sqft
    xp[1]= bath
    xp[2]=bhk
    if loc_index >= 0:
       xp[loc_index] = 1

    return lr_crf.predict([xp])[0]


In [ ]:
x.columns

In [ ]:
#Let's make price prediction for  1st phase JP Nagnar as our test with 3bathrooms  and 2 bedrooms
predict_price('1st Phase JP Nagar',1000,3,2)


In [ ]:
predict_price('1st Phase JP Nagar',1000,2,3) #with 3 bathrooms and 3 bedrooms

In [ ]:
predict_price('2nd Phase Judicial Layout',1000,3,3) #with 3 bathrooms and 3 bedrooms in  2nd Phase  Judicial Layout

In [ ]:
predict_price('Indira Nagar',1000,3,3) #with 3 bathroom and 3 bedrooms in Indira Nagar

In [ ]:
predict_price('Indira Nagar',1000,2,2) #with 2 bathrooms and 2 bedrooms in Indira Nagar

In [ ]:
predict_price('2nd Phase Judicial Layout',1000,2,3) #with 2 bathrooms and 3 bedrooms in  2nd Phase  Judicial Layout

In [ ]:
import pickle
with open('bengaluru_house_prices_model.pickle', 'wb') as f:
    pickle.dump(lr_crf, f)